In [1]:
with open("data/the-verdict.txt", 'r') as f:
    raw = f.read()

In [2]:
len(raw)

20479

In [3]:
raw[:40]

'I HAD always thought Jack Gisburn rather'

In [4]:
import re

txt = "Hello, kid. How are you doing?"
res = re.split(r"(\s)", txt)

res

['Hello,', ' ', 'kid.', ' ', 'How', ' ', 'are', ' ', 'you', ' ', 'doing?']

In [5]:
res = re.split(r'([,.!:;?_"()\']|--|\s)', txt)
res = [item.strip() for item in res if item.strip()]
res

['Hello', ',', 'kid', '.', 'How', 'are', 'you', 'doing', '?']

In [6]:
preprocessed = re.split(r'([,.!:;?_"()\']|--|\s)', raw)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
preprocessed[:15]

['I',
 'HAD',
 'always',
 'thought',
 'Jack',
 'Gisburn',
 'rather',
 'a',
 'cheap',
 'genius',
 '--',
 'though',
 'a',
 'good',
 'fellow']

In [7]:
len(preprocessed)

4690

In [8]:
# token id assignment
total_words = sorted(set(preprocessed))
vocab_size = len(total_words)

vocab_size

1130

In [9]:
vocab = {token:integer for integer, token in enumerate(total_words)}

for i, item in enumerate(vocab.items()):
    print(item)
    if i >= 20:
        break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)


In [10]:
# tokenizer class - encode and decode

class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i: s for s, i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.!:;?_"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'([,.!:;?_"()\']|--|\s)', r'\1', text)
        return text


In [11]:
tokenizer = SimpleTokenizerV1(vocab)

text = "It's the last he painted, you know!"
ids = tokenizer.encode(text)
ids

[56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 0]

In [12]:
tokenizer.decode(ids)

"It ' s the last he painted , you know !"

In [13]:
text = "how are you?"
tokenizer.encode(text)
# out of vocabulary keywords like "hello"

[560, 169, 1126, 10]

In [14]:
total_tokens = sorted(list(set(preprocessed)))
total_tokens.extend(["<|endoftext|>", "<|unk|>"])

vocab = {token:integer for integer, token in enumerate(total_tokens)}

In [15]:
len(vocab.items())

1132

In [16]:
for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)

('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


In [17]:
# tokenizer updated class - encode and decode

class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i: s for s, i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.!:;?_"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [
            item if item in self.str_to_int
            else "<|unk|>" for item in preprocessed
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'([,.!:;?_"()\']|--|\s)', r'\1', text)
        return text


In [18]:
tokenizer = SimpleTokenizerV2(vocab)

text1 = "Hello bro what's up!"
text2 = "Sunlight is mesmerising."

text = " <|endoftext|> ".join((text1, text2))
text

"Hello bro what's up! <|endoftext|> Sunlight is mesmerising."

In [19]:
tokenizer.encode(text)

[1131, 1131, 1089, 2, 850, 1051, 0, 1130, 1131, 584, 1131, 7]

In [20]:
tokenizer.decode(tokenizer.encode(text))

"<|unk|> <|unk|> what ' s up ! <|endoftext|> <|unk|> is <|unk|> ."

In [21]:
# BPE
import tiktoken

tokenizer = tiktoken.get_encoding("gpt2")

In [22]:
text = "Hello bro what's up! <|endoftext|> Sunlightismesmerising."

ids = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
ids

[15496,
 1379,
 644,
 338,
 510,
 0,
 220,
 50256,
 3825,
 2971,
 1042,
 274,
 647,
 1710,
 13]

In [23]:
tokenizer.decode(ids)

"Hello bro what's up! <|endoftext|> Sunlightismesmerising."

In [24]:
# creating input-target pairs

raw[:50]

'I HAD always thought Jack Gisburn rather a cheap g'

In [25]:
enc_text = tokenizer.encode(raw)
len(enc_text)

5145

In [26]:
sample = enc_text[:50]
sample[:5]

[40, 367, 2885, 1464, 1807]

In [27]:
context_size = 4

x = sample[:context_size]
y = sample[1:context_size + 1]


In [28]:
print(f"X: {x}")
print(f"Y:     {y}")

X: [40, 367, 2885, 1464]
Y:     [367, 2885, 1464, 1807]


In [29]:
for i in range(1, context_size+1):
    ip = sample[:i]
    t = sample[i]
    print(f"{ip} --> {t}")

[40] --> 367
[40, 367] --> 2885
[40, 367, 2885] --> 1464
[40, 367, 2885, 1464] --> 1807


In [30]:
from torch.utils.data import Dataset, DataLoader


class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))
    def __len__(self):
        return len(self.input_ids)
    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [31]:
def create_dataloader_v1(txt, batch_size=4, max_len=256,
                        stride=128, shuffle=True, drop_last=True,
                        num_workers=0):
    tokenizer = tiktoken.get_encoding('gpt2')
    dataset = GPTDatasetV1(txt, tokenizer, max_len, stride)
    dataloader = DataLoader(
        dataset,
        batch_size = batch_size,
        shuffle = shuffle,
        drop_last = drop_last,
        num_workers = num_workers
    )
    return dataloader

In [32]:
import torch
torch.__version__

'2.9.0+cu128'

In [33]:
dl = create_dataloader_v1(
    raw, batch_size=1, max_len=4, stride=1, shuffle=False
)

data_iter = iter(dl)

In [34]:
next(data_iter)

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]

In [35]:
next(data_iter)

[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]

In [36]:
dl = create_dataloader_v1(
    raw, batch_size=8, max_len=4, stride=4, shuffle=False
)

data_iter = iter(dl)
ip,t = next(data_iter)

In [37]:
print(f"Inputs:\n{ip}")
print(f"Target:\n{t}")

Inputs:
tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])
Target:
tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])


In [38]:
# token embeddings
sample_ip_id = torch.tensor([2, 4, 3, 1])

In [39]:
vocab_size = 6
output_dim = 3

embedding_layer = torch.nn.Embedding(vocab_size, output_dim)
embedding_layer.weight

Parameter containing:
tensor([[ 1.0281,  0.0552,  0.4852],
        [-0.7863, -1.7132, -0.5370],
        [ 2.3906, -1.7816, -0.9334],
        [ 1.6461, -0.3220,  0.2286],
        [-0.1663, -0.3135,  0.4420],
        [-0.8006,  0.6517,  0.1187]], requires_grad=True)

In [40]:
embedding_layer(torch.tensor(sample_ip_id))

/tmp/ipykernel_5892/3818088080.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  embedding_layer(torch.tensor(sample_ip_id))


tensor([[ 2.3906, -1.7816, -0.9334],
        [-0.1663, -0.3135,  0.4420],
        [ 1.6461, -0.3220,  0.2286],
        [-0.7863, -1.7132, -0.5370]], grad_fn=<EmbeddingBackward0>)

In [41]:
print(f"Inputs:\n{ip}")
ip.shape

Inputs:
tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])


torch.Size([8, 4])

In [42]:
vocab_size = 50257
output_dim = 256

token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

In [43]:
token_embedding = token_embedding_layer(ip)
token_embedding.shape

torch.Size([8, 4, 256])

In [46]:
# positional encoding
max_len  = 4
context_len = max_len
pos_embedding_layer = torch.nn.Embedding(context_len, output_dim)

In [47]:
pos_embedding = pos_embedding_layer(torch.arange(max_len))
pos_embedding.shape

torch.Size([4, 256])

In [48]:
input_embedding = token_embedding + pos_embedding
input_embedding.shape

torch.Size([8, 4, 256])